## Датасет

In [11]:
!ls /home/jupyter/datasphere/datasets/newspapers
import pandas as pd
df = pd.read_json("/home/jupyter/datasphere/datasets/newspapers/newspapers1934-39.json")
df

newspapers1934-39.json


,newspaper_year,newspaper_date,origin_img,newspaper_title,newspaper_url,newspaper_page_text
0,1934,01-01-1934,http://electrohelp.ebook.nekrasovka.ru/books/n...,Вечерняя Москва/1934/№ 1 (3030)/1.jpg,https://electro.nekrasovka.ru/books/6159247,"ПРОЛЕТАРИЙ ВСЕХ`СТРАН, СОЕЛИНЯЙТЕСЬ,\n\t\tПОНЕ..."
1,1934,01-01-1934,http://electrohelp.ebook.nekrasovka.ru/books/n...,Вечерняя Москва/1934/№ 1 (3030)/2.jpg,https://electro.nekrasovka.ru/books/6159247,СЕГОДНЯ |\n\t=== | января\n\tМосжилтрест об’ед...
2,1934,01-01-1934,http://electrohelp.ebook.nekrasovka.ru/books/n...,Вечерняя Москва/1934/№ 1 (3030)/3.jpg,https://electro.nekrasovka.ru/books/6159247,_ '>ЗЕЛО\nБыло —\nЗАНЯТНО«\n\tПО КНИЖНЫМ\nМАГА...
3,1934,01-01-1934,http://electrohelp.ebook.nekrasovka.ru/books/n...,Вечерняя Москва/1934/№ 1 (3030)/4.jpg,https://electro.nekrasovka.ru/books/6159247,отеавиовый звуковой к. ВНИИ ГЛАНИ Ю ДЛЯ ОБСЛУЖ...
4,1934,03-01-1934,http://electrohelp.ebook.nekrasovka.ru/books/n...,Вечерняя Москва/1934/№ 3 (3032)/1.jpg,https://electro.nekrasovka.ru/books/6159275,НИ ВСЕХ СТРАН. СОЕДИНЯЙТЕСЬ!\n\tЯНВАРЯ 1934 г....
...,...,...,...,...,...,...
15913,1939,30-12-1939,http://electrohelp.ebook.nekrasovka.ru/books/s...,"Правда/1939/№ 360 (8045), 30 декабря/2.jpg",https://electro.nekrasovka.ru/books/6260565,None
15914,1939,30-12-1939,http://electrohelp.ebook.nekrasovka.ru/books/s...,"Правда/1939/№ 360 (8045), 30 декабря/3.jpg",https://electro.nekrasovka.ru/books/6260565,None
15915,1939,30-12-1939,http://electrohelp.ebook.nekrasovka.ru/books/s...,"Правда/1939/№ 360 (8045), 30 декабря/4.jpg",https://electro.nekrasovka.ru/books/6260565,None
15916,1939,30-12-1939,http://electrohelp.ebook.nekrasovka.ru/books/s...,"Правда/1939/№ 360 (8045), 30 декабря/5.jpg",https://electro.nekrasovka.ru/books/6260565,None


## Импорты

In [12]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import base64
import random
import numpy as np
import cv2
import re
from yandex_cloud_ml_sdk import YCloudML
import spacy
from pathlib import Path
spacy.cli.download("ru_core_news_lg")
nlp = spacy.load("ru_core_news_lg")

folder_id = os.environ['folder_id']
api_key = os.environ['api_key']

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 18.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## API

In [3]:
def call_api(url, data):
    headers = { "Authorization" : f"Api-Key {api_key}" }
    return requests.post(url, json=data, headers=headers).json()

In [4]:
def call_api_get(url, data):
    headers = { "Authorization" : f"Api-Key {api_key}" }
    return requests.get(url, headers=headers).json()

## OCR

In [5]:
def ocr(img):
    buffer = BytesIO()
    img.save(buffer,format="JPEG")
    myimage = buffer.getvalue()

    j = {
      "mimeType": "JPEG",
      "languageCodes": ["*"],
      "model": "page-column-sort",
      "content": base64.b64encode(myimage).decode('utf-8')
    }
    res = call_api("https://ocr.api.cloud.yandex.net/ocr/v1/recognizeText",j)
    return res

## Разбиение на столбцы

In [6]:
def process_image(img_array):
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    
    pixel_values = gray.flatten().astype(np.float64)
    p_low, p_high = np.percentile(pixel_values, [8, 92])
    filtered_pixels = pixel_values[(pixel_values >= p_low) & (pixel_values <= p_high)]
    
    mean_brightness_filtered = np.mean(filtered_pixels) - 15
    
    thresh_img = cv2.threshold(gray, float(mean_brightness_filtered), 255, cv2.THRESH_BINARY_INV)[1]
    
    kernel_dilate = np.ones((31, 5), np.uint8)
    thresh_img = cv2.dilate(thresh_img, kernel_dilate, iterations=1)
    
    kernel_erode = np.ones((1, 17), np.uint8)
    thresh_img = cv2.erode(thresh_img, kernel_erode, iterations=1)

    image_width = thresh_img.shape[1]
    image_height = thresh_img.shape[0]
    edge_margin_x = int(image_width * 0.05)
    edge_margin_y = int(image_height * 0.05)
    
    roi = thresh_img[edge_margin_y:image_height-edge_margin_y, edge_margin_x:image_width-edge_margin_x]
    
    vertical_projection_roi = np.sum(roi, axis=0) / roi.shape[0]
    
    peak_threshold = np.max(vertical_projection_roi) * 0.4
    peak_indices = np.where(vertical_projection_roi > peak_threshold)[0]
    
    if len(peak_indices) > 0:
        first_content_idx = peak_indices[0]
        last_content_idx = peak_indices[-1]
        
        content_values = vertical_projection_roi[first_content_idx:last_content_idx+1]
        initial_threshold = np.mean(content_values) * 0.4
    else:
        initial_threshold = np.mean(vertical_projection_roi) * 0.5
    
    upper_values = vertical_projection_roi[vertical_projection_roi > initial_threshold]
    lower_values = vertical_projection_roi[vertical_projection_roi < initial_threshold]
    
    if len(upper_values) > 0 and len(lower_values) > 0:
        upper_mean = np.mean(upper_values)
        lower_mean = np.mean(lower_values)
        mid_threshold = (upper_mean + lower_mean) / 2
    else:
        mid_threshold = initial_threshold
    
    threshold = mid_threshold
    
    vertical_projection = np.zeros(image_width)
    vertical_projection[edge_margin_x:image_width-edge_margin_x] = vertical_projection_roi
    
    column_separators = np.where(vertical_projection < threshold)[0]
    
    column_boundaries = []
    if len(column_separators) > 0:
        current_group_start = column_separators[0]
        
        for i in range(1, len(column_separators)):
            if column_separators[i] > column_separators[i-1] + 1:
                mid_point = (current_group_start + column_separators[i-1]) // 2
                column_boundaries.append(mid_point)
                current_group_start = column_separators[i]
        
        mid_point = (current_group_start + column_separators[-1]) // 2
        column_boundaries.append(mid_point)
    
    filtered_boundaries = []
    if column_boundaries:
        image_width = thresh_img.shape[1]
        edge_margin = int(image_width * 0.05)
        min_distance = int(image_width * 0.03)
        
        boundaries_no_edges = [b for b in column_boundaries if b > edge_margin and b < (image_width - edge_margin)]
        
        if boundaries_no_edges:
            filtered_boundaries.append(boundaries_no_edges[0])
            for i in range(1, len(boundaries_no_edges)):
                if boundaries_no_edges[i] - filtered_boundaries[-1] < min_distance:
                    filtered_boundaries[-1] = (filtered_boundaries[-1] + boundaries_no_edges[i]) // 2
                else:
                    filtered_boundaries.append(boundaries_no_edges[i])
    
    filtered_boundaries.insert(0, 0)
    filtered_boundaries.append(image_width)
    
    column_images = []
    for i in range(len(filtered_boundaries)-1):
        column_image = img_array[:, filtered_boundaries[i]:filtered_boundaries[i + 1]]
        column_images.append(column_image)
    
    return column_images

## Этика

In [7]:
def neutralize_with_spacy(text):
    doc = nlp(text)
    replacements = {}
    neutralized = text
    for ent in doc.ents:
        if ent.label_ in ["ORG", "PER", "LOC", "DATE"]:
            entity_id = f"[{ent.label_}_{len(replacements) + 1}]"
            replacements[entity_id] = ent.text
            neutralized = neutralized.replace(ent.text, entity_id, 1)
    
    return neutralized, replacements

def encode_from_spacy(text, replacements):
    for entity_id, original_text in replacements.items():
        text = text.replace(entity_id, original_text)
    return text

## Промт для этики

In [8]:
prompt = """
[ТЕХНИЧЕСКАЯ ОБРАБОТКА ИСТОРИЧЕСКИХ ДОКУМЕНТОВ]
Ты - программный инструмент для реконструкции текстов, которому на вход подается распознанный с помощью OCR текст газетного листа. 

Твоя задача:
1. Текстовая реконструкция:
   - Сохранить исходный текст с минимальными изменениями
   - Исправить только явные ошибки OCR (опечатки, пропущенные буквы, слитные слова)
   - Сохранить оригинальную стилистику и форматирование

2. Разделение на статьи:
   - Четко разделить текст на отдельные статьи по следующим признакам:
     * Заголовки (выделенные шрифтом или положением)
     * Тематические разрывы в тексте
     * Видимые разделители (например, "====" или пустые строки)
     * Логические смысловые границы
   - Между статьями вставлять явный разделитель: "\n\n====СТАТЬЯ====\n\n"
   - Сохранять оригинальные заголовки статей

Правила обработки:
- НЕ интерпретировать содержание
- НЕ добавлять свою аналитику
- НЕ изменять исторические названия и термины
- НЕ объединять разорванные строки, если это не явная ошибка OCR

Формат вывода:
- Каждая статья должна быть отдельным элементом в списке
- Статьи должны идти в том же порядке, что и в оригинале
- Сохранять все оригинальные абзацы внутри статей
"""

## Ключи

In [16]:
folder_id = os.environ['folder_id']
api_key = os.environ['api_key']
sdk = YCloudML(folder_id=folder_id, auth=api_key)
model = sdk.models.completions("yandexgpt", model_version="rc")
df = pd.read_json("/home/jupyter/datasphere/datasets/newspapers/newspapers1934-39.json")

## Сохранение статей в txt

In [44]:
!rm -rf /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/

In [51]:
s = []
for i in range(1, 13):
    for j in range(9, 31):
        month = f"{i}"
        day = f"{j}"
        if (j < 10):
            day = f"0{j}"
        if (i < 10):
            month = f"0{i}"
        s.append(f"{day}-{month}-1935")


In [ ]:
index = -1
newspapers_texts = {}

for date in s:
    print(date)
    for page in df[df['newspaper_date'] == date].index:
        index += 1
        print(f"Дата {date}, обработка строки {page}, порядковый номер {index}")
        url = df.iloc[page]['origin_img']
        response = requests.get(url)
        img_array = np.array(Image.open(BytesIO(response.content)))
        column_images = process_image(img_array)
        
        year = str(df.iloc[page]['newspaper_date'])
        title_part = df.iloc[page]['newspaper_title'].split('/')[0].strip()
        safe_title = re.sub(r'[^\w\s-]', '', title_part).strip()
        safe_title = re.sub(r'[-\s]+', '_', safe_title)
        
        newspaper_key = f"{year}_{safe_title}"
        
        if newspaper_key not in newspapers_texts:
            newspapers_texts[newspaper_key] = ""
        
        save_dir = "/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content"
        base_save_dir = Path(f"{save_dir}/{year}/{safe_title}")
        base_save_dir.mkdir(parents=True, exist_ok=True)
        
        for col_idx, column_img in enumerate(column_images, start=1):
            try:
                column_pil = Image.fromarray(column_img)
                ocr_result = ocr(column_pil)
                
                if 'result' not in ocr_result or 'textAnnotation' not in ocr_result['result']:
                    print(f"Предупреждение: некорректный ответ OCR для колонки {col_idx}")
                    continue
                else:
                    recognized_text = ocr_result['result']['textAnnotation']['fullText']
                
                neutralize_text, replacement = neutralize_with_spacy(recognized_text)
                response_before_encode = model.run([
                    {"role": "system", "text": prompt},
                    {"role": "user", "text": neutralize_text}
                ])
                response = encode_from_spacy(response_before_encode.text, replacement)
                
                articles = response.split('\n\n====СТАТЬЯ====\n\n')
                
                for art_idx, article in enumerate(articles, start=1):
                    if article.strip() and not article.endswith("(https://ya.ru)"):
                        article_file_path = base_save_dir / f"{index}_col{col_idx}_article{art_idx}.txt"
                        try:
                            with open(article_file_path, 'w', encoding='utf-8') as f:
                                f.write(article.strip())
                            print(f"Файл статьи сохранен: {article_file_path}")
                        except Exception as e:
                            print(f"Ошибка при сохранении файла статьи: {e}")
                
                newspapers_texts[newspaper_key] += response + "\n\n"
                
            except Exception as ocr_error:
                print(f"Ошибка при обработке колонки {col_idx}: {str(ocr_error)}")
                error_file_path = base_save_dir / f"{index}_col{col_idx}_error.txt"
                with open(error_file_path, 'w', encoding='utf-8') as f:
                    f.write(f"[Ошибка при обработке колонки: {str(ocr_error)}]")

09-01-1935
Дата 09-01-1935, обработка строки 776, порядковый номер 0
Файл статьи сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/09-01-1935/Вечерняя_Москва/0_col1_article1.txt
Файл статьи сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/09-01-1935/Вечерняя_Москва/0_col1_article2.txt
Файл статьи сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/09-01-1935/Вечерняя_Москва/0_col1_article3.txt
Файл статьи сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/09-01-1935/Вечерняя_Москва/0_col7_article1.txt
Файл статьи сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/09-01-1935/Вечерняя_Москва/0_col7_article2.txt
Файл статьи сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/09-01-1935/Вечерняя_Москва/0_col7_article3.txt
Дата 09-01-1935, обработка строки 777, порядковый номер 1
Файл статьи сох

KeyboardInterrupt: 

## Создание названия статьи и основных мыслей

In [38]:
import json
def create_topic(path: str):
    with open(path, 'r', encoding='utf-8') as file:
        data = file.read().strip()

    prompt_for_summary = '''
    [ТЕХНИЧЕСКАЯ ОБРАБОТКА ИСТОРИЧЕСКИХ ДОКУМЕНТОВ]
    Ты - программный инструмент для реконструкции текстов, которому на вход подается распознанный текст ИСТОРИЧЕСКОЙ статьи. 

    Твоя задача:
    1. Придумай название текста в паре слов
    2. Верни ТОЛЬКО его

    Правила обработки:
    - НЕ интерпретировать содержание
    - НЕ добавлять свою аналитику
    - НЕ изменять исторические названия и термины
    - НЕ объединять разорванные строки, если это не явная ошибка OCR
    '''

    neutralize_text, replacement = neutralize_with_spacy(data)
    response_before_encode = model.run([
        {"role": "system", "text": prompt_for_summary},
        {"role": "user", "text": neutralize_text[:(len(neutralize_text)//3)]}
    ])
    response = encode_from_spacy(response_before_encode.text, replacement)
    if (response.endswith("(https://ya.ru)")):
        return None
    return response

def create_summary(path):
    with open(path, 'r', encoding='utf-8') as file:
        data = file.read().strip()

    prompt_for_summary = '''
    [ТЕХНИЧЕСКАЯ ОБРАБОТКА ИСТОРИЧЕСКИХ ДОКУМЕНТОВ]
    Ты - программный инструмент для реконструкции текстов, которому на вход подается распознанный текст ИСТОРИЧЕСКОЙ статьи. 

    Твоя задача:
    1. Придумай основную мысль текста в нескольких предложениях
    2. Верни ТОЛЬКО его

    Правила обработки:
    - НЕ интерпретировать содержание
    - НЕ добавлять свою аналитику
    - НЕ изменять исторические названия и термины
    - НЕ объединять разорванные строки, если это не явная ошибка OCR
    '''

    neutralize_text, replacement = neutralize_with_spacy(data)
    response_before_encode = model.run([
        {"role": "system", "text": prompt_for_summary},
        {"role": "user", "text": neutralize_text[:(len(neutralize_text)//3)]}
    ])
    
    response = encode_from_spacy(response_before_encode.text, replacement)
    if (response.endswith("(https://ya.ru)")):
        return None
    return response

None


## Сохранение статей в jsonы

In [ ]:
import json
import re
from pathlib import Path

def process_file(txt_path, output_dir):
    with open(txt_path, 'r', encoding='utf-8') as f:
        content = f.read().strip()
    
    if not content:
        print(f"Пустой файл: {txt_path}")
        return
    
    parts = txt_path.parts
    try:
        date = parts[-3]
        source = parts[-2]
        source = source.replace('_', ' ')
    except IndexError:
        print(f"Не удалось извлечь дату и источник из пути: {txt_path}")
        return
    
    try:
        print(txt_path)
        topic = create_topic(txt_path)
        summary = create_summary(txt_path)
        entities_response = model.run([{"role": "system", "text": """Извлеки ключевые имена, организации и термины из текста. 
            Ответь строго в формате: "сущность1, сущность2, сущность3". 
            Только конкретные имена, организации и термины."""},
            {"role": "user", "text": content}])

        entities_str = entities_response.text.strip()
        entities = [e.strip() for e in entities_str.split(',') if e.strip()]
    except Exception as e:
        print(f"Ошибка при работе с моделью для файла {txt_path}: {e}")
        return
    
    article_json = {
        "date": date,
        "source": source,
        "topic": topic,
        "summary": summary,
        "content": content,
        "entities": entities
    }
    
    relative_path = txt_path.relative_to(Path("/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content"))
    # print(os.path.basename(relative_path.with_suffix('.json')))
    json_path = output_dir / os.path.basename(relative_path.with_suffix('.json'))
    json_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(article_json, f, ensure_ascii=False, indent=2)
    
    print(f"JSON сохранен: {json_path}")

def process_directory(input_dir="/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content", 
                      output_dir="/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/nodes"):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    output_path.mkdir(parents=True, exist_ok=True)
    
    txt_files = list(input_path.glob('**/*.txt'))
    if not txt_files:
        print("Не найдено txt файлов для обработки.")
        return
    print(f"Найдено {len(txt_files)} файлов для обработки...")
    
    for txt_file in txt_files:
        try:
            process_file(txt_file, output_path)
        except Exception as e:
            print(f"Ошибка при обработке файла {txt_file}: {e}")

if __name__ == "__main__":
    process_directory()

Найдено 9508 файлов для обработки...
/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/01-01-1935/Вечерняя_Москва/0_col2_article1.txt
JSON сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/nodes/0_col2_article1.json
/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/01-01-1935/Вечерняя_Москва/0_col2_article2.txt
JSON сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/nodes/0_col2_article2.json
/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/01-01-1935/Вечерняя_Москва/0_col3_article1.txt
JSON сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/nodes/0_col3_article1.json
/home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/content/01-01-1935/Вечерняя_Москва/0_col3_article2.txt
JSON сохранен: /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/nodes/0_col3_article2.json
/home/jupyter/datasphere/s3/mipt-nekrasovka/dat

In [66]:
!rm -rf /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/nodes


In [67]:
ls /home/jupyter/datasphere/s3/mipt-nekrasovka/data/articles_dataset/


content/
nodes_mozzhukhin/
